In [63]:
!pip install xgboost

In [141]:
import warnings
warnings.filterwarnings('ignore')

In [142]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [143]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_turns = pd.read_csv('turns.csv')
df_game = pd.read_csv('games.csv')

In [144]:
df_train.head()

,game_id,nickname,score,rating
0,1,BetterBot,335,1637
1,1,stevy,429,1500
2,3,davidavid,440,1811
3,3,BetterBot,318,2071
4,4,Inandoutworker,119,1473


In [145]:
df_test.head()

,game_id,nickname,score,rating
0,2,Super,488,NaN
1,2,BetterBot,401,2000.0
2,7,STEEBot,377,2082.0
3,7,Priya1,379,NaN
4,11,STEEBot,334,1829.0


In [146]:
df_game.head()

,game_id,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
2,3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
4,5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722


In [147]:
df_turns.head()

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play


In [148]:
def fill_na(column_name,d_type):
    if d_type == 'train':
        elements = df_train[column_name].dropna().unique()
        proba = [df_train[column_name].value_counts()[i] / len(df_train[column_name].dropna()) for i in elements]
        df_train[column_name].fillna(np.random.choice(a=elements,p=proba),inplace=True)
    elif d_type == 'test':
        elements = df_test[column_name].dropna().unique()
        proba = [df_test[column_name].value_counts()[i] / len(df_test[column_name].dropna()) for i in elements]
        df_test[column_name].fillna(np.random.choice(a=elements,p=proba),inplace=True)
    elif d_type == 'turn':
        elements = df_turns[column_name].dropna().unique()
        proba = [df_turns[column_name].value_counts()[i] / len(df_turns[column_name].dropna()) for i in elements]
        df_turns[column_name].fillna(np.random.choice(a=elements,p=proba),inplace=True)
    else:
        elements = df_game[column_name].dropna().unique()
        proba = [df_game[column_name].value_counts()[i] / len(df_game[column_name].dropna()) for i in elements]
        df_game[column_name].fillna(np.random.choice(a=elements,p=proba),inplace=True)

In [149]:
for i in df_turns.columns:
    if df_turns[i].isna().sum() / len(df_turns) > 0:
        print(i)
        print(df_turns[i].value_counts())

rack
rack
I          2900
U          1883
E          1780
R          1772
N          1655
           ... 
?EOPRTZ       1
BEGLSSU       1
ABBESSU       1
LLNQ          1
ABHLLNQ       1
Name: count, Length: 626752, dtype: int64
location
location
15A    24190
H1     23894
8D     23499
O1     23377
8G     22077
       ...  
E14      178
13N      141
3N       121
M14      103
C14       86
Name: count, Length: 418, dtype: int64
move
move
--          14302
QI          14221
Q.          10318
-            8065
.I           5167
            ...  
COND.RS         1
AiLMEN.S        1
BEShINE         1
sPAMMED         1
GADJO.          1
Name: count, Length: 356847, dtype: int64
turn_type
turn_type
Play             1872864
End                67987
Exchange           48997
Pass                8065
Timeout             4143
Challenge           2571
Six-Zero Rule        476
Name: count, dtype: int64


In [150]:
for column in df_train.isna().sum()[df_train.isna().sum() > 0].index:
    fill_na(column,'train')

In [151]:
df_turns.drop('move',axis=1,inplace=True)
df_turns.drop('rack',axis=1,inplace=True)

In [152]:
for column in df_turns.isna().sum()[df_turns.isna().sum() > 0].index:
    fill_na(column,'turn')

In [153]:
from sklearn.preprocessing import LabelEncoder

In [154]:
df_test['rating'].fillna(1,inplace=True)

In [155]:
df_train = pd.merge(df_train, df_game, on='game_id', how='left')
df_test = pd.merge(df_test, df_game, on='game_id', how='left')

In [156]:
df_train = pd.merge(df_train, df_turns, on=['game_id', 'nickname'],how='left')
df_test = pd.merge(df_test, df_turns, on=['game_id', 'nickname'],how='left')

In [157]:
df_train.drop('nickname',axis=1,inplace=True)
df_test.drop('nickname',axis=1,inplace=True)

In [158]:
df_train.drop('first',axis=1,inplace=True)
df_test.drop('first',axis=1,inplace=True)

In [159]:
df_train.drop('created_at',axis=1,inplace=True)
df_test.drop('created_at',axis=1,inplace=True)

In [160]:
for i in df_test.columns:
    if df_train[i].dtype == object:
        if len(set(df_test[i]).difference(set(df_train[i]))) > 0:
            print(i)
            print(set(df_test[i]).difference(set(df_train[i])))

In [161]:
for i in df_test.columns:
    if df_test[i].dtype == object:
        label = LabelEncoder()
        df_train[i] = label.fit_transform(df_train[i])
        df_test[i] = label.transform(df_test[i])

In [162]:
df_test.isna().sum()

game_id                  0
score_x                  0
rating                   0
time_control_name        0
game_end_reason          0
winner                   0
lexicon                  0
initial_time_seconds     0
increment_seconds        0
rating_mode              0
max_overtime_minutes     0
game_duration_seconds    0
turn_number              0
location                 0
points                   0
score_y                  0
turn_type                0
dtype: int64

In [163]:
df = pd.concat([df_train,df_test])

In [164]:
df

,game_id,score_x,rating,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,turn_number,location,points,score_y,turn_type
0,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,1,188,10,10,4
1,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,3,162,16,26,4
2,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,5,371,28,54,4
3,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,7,144,29,83,4
4,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,9,244,68,151,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614204,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,25,0,66,335,4
614205,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,27,257,32,367,4
614206,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,29,232,19,386,4
614207,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,30,81,8,394,1


In [165]:
df[df['rating'] == 1]

,game_id,score_x,rating,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,turn_number,location,points,score_y,turn_type
0,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,1,185,24,24,4
1,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,3,280,34,58,4
2,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,5,76,44,102,4
3,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,7,349,33,135,4
4,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,9,372,94,229,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614204,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,25,0,66,335,4
614205,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,27,257,32,367,4
614206,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,29,232,19,386,4
614207,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,30,81,8,394,1


In [166]:
df_train = df[df['rating'] != 1]
df_test = df[df['rating'] == 1]

In [167]:
df_train

,game_id,score_x,rating,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,turn_number,location,points,score_y,turn_type
0,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,1,188,10,10,4
1,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,3,162,16,26,4
2,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,5,371,28,54,4
3,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,7,144,29,83,4
4,1,335,1637.0,2,2,1,3,1200,0,0,1,674.844274,9,244,68,151,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614186,72769,626,2110.0,0,2,0,0,180,0,1,1,218.043751,20,396,23,318,4
614187,72769,626,2110.0,0,2,0,0,180,0,1,1,218.043751,22,77,30,348,4
614188,72769,626,2110.0,0,2,0,0,180,0,1,1,218.043751,24,272,28,376,4
614189,72769,626,2110.0,0,2,0,0,180,0,1,1,218.043751,26,221,194,570,4


In [168]:
df_test

,game_id,score_x,rating,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,turn_number,location,points,score_y,turn_type
0,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,1,185,24,24,4
1,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,3,280,34,58,4
2,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,5,76,44,102,4
3,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,7,349,33,135,4
4,2,488,1.0,2,2,1,0,3600,0,1,1,364.214418,9,372,94,229,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614204,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,25,0,66,335,4
614205,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,27,257,32,367,4
614206,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,29,232,19,386,4
614207,72769,384,1.0,0,2,0,0,180,0,1,1,218.043751,30,81,8,394,1


In [169]:
df_train = df_train.drop_duplicates(subset='game_id')
df_test = df_test.drop_duplicates(subset='game_id')

In [170]:
df_sub = pd.read_csv('sample_submission.csv')

In [171]:
df_sub

,game_id,rating
0,2,1790.322674
1,7,1790.322674
2,11,1790.322674
3,14,1790.322674
4,27,1790.322674
...,...,...
22358,72760,1790.322674
22359,72761,1790.322674
22360,72762,1790.322674
22361,72768,1790.322674


In [172]:
df_train.drop('game_id',axis=1,inplace=True)
df_test.drop('game_id',axis=1,inplace=True)

In [173]:
df_test.drop('rating',axis=1,inplace=True)

In [174]:
X_train = df_train.drop('rating',axis=1)
X_test = df_test.copy()
y_train = df_train['rating']

# RandomForestRegressor

In [327]:
from sklearn.ensemble import RandomForestRegressor

In [328]:
model = RandomForestRegressor()

In [329]:
from sklearn.model_selection import GridSearchCV

In [333]:
params = {
    'n_estimators' : [80,100],
    'criterion' : ["squared_error", "friedman_mse", "poisson"],
    'bootstrap' : [True,False],
    'oob_score' : [True,False]
}

In [334]:
final_model = GridSearchCV(model, param_grid=params, cv=3, verbose=2)

In [335]:
final_model.fit(X_train,y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END bootstrap=True, criterion=squared_error, n_estimators=80, oob_score=True; total time=  30.6s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=80, oob_score=True; total time=  30.3s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=80, oob_score=True; total time=  29.0s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=80, oob_score=False; total time=  29.6s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=80, oob_score=False; total time=  29.6s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=80, oob_score=False; total time=  27.9s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=100, oob_score=True; total time=  37.9s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=100, oob_score=True; total time=  40.2s
[CV] END bootstrap=True, criterion=squared_error, n_estimators=100, oob_score=True; total time=  36.6s
[CV] END bootst

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['squared_error', 'friedman_mse',
                                       'poisson'],
                         'n_estimators': [80, 100],
                         'oob_score': [True, False]},
             verbose=2)

In [336]:
final_model.best_params_

{'bootstrap': True,
 'criterion': 'friedman_mse',
 'n_estimators': 100,
 'oob_score': True}

In [337]:
y_pred = final_model.predict(X_test)

In [338]:
df_sub['rating'] = y_pred

In [339]:
df_sub.to_csv('sub.csv',index=False,index_label=False)

In [285]:
df_train.corr()

,score_x,rating,rating_mode,max_overtime_minutes,points,score_y
score_x,1.000000,0.429757,0.097528,-0.076316,0.259940,0.259940
rating,0.429757,1.000000,0.334834,-0.398088,0.148007,0.148007
rating_mode,0.097528,0.334834,1.000000,-0.453328,0.044545,0.044545
max_overtime_minutes,-0.076316,-0.398088,-0.453328,1.000000,-0.049453,-0.049453
points,0.259940,0.148007,0.044545,-0.049453,1.000000,1.000000
score_y,0.259940,0.148007,0.044545,-0.049453,1.000000,1.000000


In [47]:
df_train.drop('time_control_name',axis=1,inplace=True)
df_test.drop('time_control_name',axis=1,inplace=True)

In [48]:
df_train.drop('lexicon',axis=1,inplace=True)
df_test.drop('lexicon',axis=1,inplace=True)

In [49]:
df_train.drop('game_end_reason',axis=1,inplace=True)
df_test.drop('game_end_reason',axis=1,inplace=True)

In [50]:
df_train.drop('winner',axis=1,inplace=True)
df_test.drop('winner',axis=1,inplace=True)

In [51]:
df_train.drop('initial_time_seconds',axis=1,inplace=True)
df_test.drop('initial_time_seconds',axis=1,inplace=True)

In [52]:
df_train.drop('increment_seconds',axis=1,inplace=True)
df_test.drop('increment_seconds',axis=1,inplace=True)

In [53]:
df_train.drop('game_duration_seconds',axis=1,inplace=True)
df_test.drop('game_duration_seconds',axis=1,inplace=True)

In [54]:
df_train.drop('turn_number',axis=1,inplace=True)
df_test.drop('turn_number',axis=1,inplace=True)

In [55]:
df_train.drop('location',axis=1,inplace=True)
df_test.drop('location',axis=1,inplace=True)

In [56]:
df_train.drop('turn_type',axis=1,inplace=True)
df_test.drop('turn_type',axis=1,inplace=True)

# XGBoost

In [175]:
from xgboost import XGBRegressor

In [176]:
model = XGBRegressor()

In [177]:
from sklearn.model_selection import GridSearchCV

In [210]:
params = {
    'n_estimators' : [60,65,70]
}

In [211]:
final_model = GridSearchCV(model,param_grid=params,cv=5,verbose=2)

In [212]:
final_model.fit(X_train,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ....................................n_estimators=60; total time=   0.0s
[CV] END ....................................n_estimators=60; total time=   0.0s
[CV] END ....................................n_estimators=60; total time=   0.0s
[CV] END ....................................n_estimators=60; total time=   0.0s
[CV] END ....................................n_estimators=60; total time=   0.0s
[CV] END ....................................n_estimators=65; total time=   0.0s
[CV] END ....................................n_estimators=65; total time=   0.0s
[CV] END ....................................n_estimators=65; total time=   0.0s
[CV] END ....................................n_estimators=65; total time=   0.0s
[CV] END ....................................n_estimators=65; total time=   0.0s
[CV] END ....................................n_estimators=70; total time=   0.0s
[CV] END ....................................n_es

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             param_grid={'n_estimators': [60, 65, 70]}, verbose=2)

In [213]:
final_model.best_params_

{'n_estimators': 70}

In [214]:
y_pred = final_model.predict(X_test)

In [215]:
df_sub = pd.read_csv('sample_submission.csv')

In [216]:
df_sub['rating'] = y_pred

In [217]:
df_sub.to_csv('sub.csv',index=False,index_label=False)